# Postprocessing

This notebook contains the postprocessing workflow for the shallow-water simulations generated by `scripts/simulation.jl`.  
Its purpose is to extract dynamically relevant diagnostics from the raw model output and to prepare fields for analysis and visualization used in the associated study.

Input–output utilities (loading model output, metadata, and derived fields) are implemented in `utils/io.py`.  
Methods for interpolating simulation fields onto
- constant-$H$ (depth-following) contours, and
- constant-$y$ (straight cross-slope transects)

are provided in `utils/grid.py`.

The focus of this notebook is on computing and organizing these postprocessed quantities in a reproducible way.  
Final figures and publication-ready visualizations based on this output are produced in the companion notebook `make_figures.ipynb`.



In [1]:
# Import necessary libraries
import xarray as xr
import numpy as np
from pathlib import Path
import sys
import json

# Set up project root for imports
project_root = Path.cwd().parents[0]
sys.path.append(str(project_root))

# Custom utilities
from utils.config import load_config
from utils.io import read_raw_output, ensure_dir
from utils.grid import prepare_dsH, interp_ds, mean_onH

In [2]:
# output folder for postprocessed data
output_folder = str(project_root)+"/output/processed/"


# Focus depth contour. Used for plotting and analysis. 
# This correspond to the mid-depth contour
focus_j = 45


# subregion for analysis
analysis_region = {
    # short forcing period
    "short": {
        "focus_time_start" : -(128+8)*8,
        "focus_time_end"   : -8*8,
        "focus_j_start": 20,
        "focus_j_end": 70,
    },
    # long forcing period
    "long": {
        "focus_time_start" : -(128+64)*8,
        "focus_time_end"   : -64*8,
        "focus_j_start": 20,
        "focus_j_end": 70,
    }
}


def calculate_analytical_estimates_xr(ds,forcing_vars,params):
    """
    Analytical estimate of the response y to forcing F with exponential decay time scale H(j)/R. 
    F can be either a single variable or a sum of multiple variables. 
    So either just surface stress, or surface stress + relative vorticity forcing, etc.
    The analytical solution is computed as a discrete convolution in time:
    
    y(t_j) = ∫_0^{t_j} exp[-(R/H(j)) * (t_j - τ)] * F(τ, j) dτ
    
    Arguments:
    - ds: xarray.Dataset with model output data interpolated to H contours. Must contain "depth" and "time" dimensions.
    - forcing_vars: list of variable names in ds to sum as forcing F.
    - params: dictionary with model parameters, must include "R" and "outputtime".  
    
    Returns:
    - y: xarray.DataArray with the analytical estimate of the response,
         with same dimensions as the summed forcing F.

    """
    R = float(params["R"])
    dt = float(params["outputtime"]) 

    H = ds["depth"]  
    
    # Sum forcing fields (must have dims include "time" and "j")
    F = sum(ds[v] for v in forcing_vars)

    t = ds["time"]
    nT = t.sizes["time"]
    if nT == 0:
        raise ValueError("Empty time axis.")
    if nT == 1:
        return xr.zeros_like(F)

    # k(j) = R/H(j), broadcast over all non-time dims of F
    k = (R / H).broadcast_like(F.isel(time=0))

    # Precompute coefficients (constant in time)
    decay = np.exp(-k * dt)
    # Safe alpha for k≈0: limit -> dt
    alpha = xr.where(np.abs(k) > 0, (1.0 - decay) / k, dt)

    # build y 
    y0 = xr.zeros_like(F.isel(time=0, drop=True))
    ys = [y0]
    for i in range(1, nT):
        Fi_1 = F.isel(time=i - 1, drop=True)  # drop time here too
        y_next = decay * ys[-1] + alpha * Fi_1
        ys.append(y_next)
    y = xr.concat(ys, dim="time").assign_coords(time=t) / H

    return y

## Focus cases

We begin by postprocessing the *focus cases*: shallow-water simulations forced by spatially uniform along-slope wind stress over a corrugated slope bathymetry. Two forcing periods are considered:
- a long-period case (128 days), and
- a short-period case (16 days).

These two cases are intended to highlight differences in the dynamical response across forcing timescales.

For each case, the workflow consists of two complementary parts:

1. **Depth-following (constant-$H$) diagnostics**, used to evaluate circulation and momentum balances along topographically steered flow paths. In this framework, circulation is averaged along depth contours, and momentum terms are expressed in a contour-integrated form consistent with the theoretical framework used in the paper. Time series at a representative mid-slope contour are extracted and compared to linear and nonlinear analytical estimates.

2. **Cartesian (constant-$y$) diagnostics**, used to compute momentum terms on straight along-slope transects. These diagnostics provide access to terms such as topographic form stress and momentum flux convergence, which are not explicitly present in the depth-following formulation but are useful for interpretation and comparison.

For each case, we:
- interpolate the raw model output to the required coordinate system,
- compute the relevant momentum terms,
- restrict the analysis to a dynamically equilibrated focus period, and
- save compact NetCDF files containing only the quantities needed for subsequent analysis and figure generation.

The loop below implements this workflow for the long- and short-period forcing cases.


In [3]:
for case in ["long", "short"]:
    # load data
    params = load_config(f"../configs/baseline_forcing/{case}.json")
    ds = read_raw_output(params)
    

    #####################################################
    #### DEPTH-FOLLOWING INTERPOLATION AND DIAGNOSES ####
    #####################################################
    
    ### interpolate to depth-following grid ###
    # determine target depths H_targets (mean depth along xC at each yC)
    H_targets = ds.bath.mean("xC").values
    
    dsH = prepare_dsH(ds, params, H_targets)
    
    ### calculate circulation ###
    dsH["circulation"] = mean_onH(dsH, variable="ui")
        
    ### calculate momentum terms ###
    dsH["BS"] = -dsH.circulation*params["R"]                       # Bottom stress
    dsH["RVF"] = mean_onH(dsH, variable="zetaflux") * dsH.depth    # Relative vorticity flux
    dsH["SS"] = mean_onH(dsH, variable="forcing_i")                # Surface stress

    ### timeseries at focus depth contour ###
    
    # circulation estimates
    dsH["linear_estimate"] = calculate_analytical_estimates_xr(dsH, ["SS"], params)
    dsH["nonlinear_estimate"] = calculate_analytical_estimates_xr(dsH, ["SS", "RVF"], params)
    
    ts = xr.Dataset()
    ts["circulation"] = dsH["circulation"].isel(j=focus_j)
    ts["linear_estimate"] = dsH["linear_estimate"].isel(j=focus_j)
    ts["nonlinear_estimate"] = dsH["nonlinear_estimate"].isel(j=focus_j)
    
    # select focus period
    ts = ts.isel(
        time=slice(
            analysis_region[case]["focus_time_start"],
            analysis_region[case]["focus_time_end"],
        ),
    )
    ts["time"] = ts["time"] - ts["time"].values[0]
    
    ensure_dir(output_folder+"/timeseries/")
    ts.squeeze().to_netcdf(output_folder+f"/timeseries/analytical_estimates_{case}.nc")
    
    
    # select focus region and period
    dsH = dsH.isel(
        time=slice(
            analysis_region[case]["focus_time_start"],
            analysis_region[case]["focus_time_end"],
        ),
        j=slice(
            analysis_region[case]["focus_j_start"],
            analysis_region[case]["focus_j_end"],
        ),
    )
    dsH["time"] = dsH["time"] - dsH["time"].values[0]  # reset time to start at 0
    
    
    ### save relevant momentum terms ###
    if case == "short":
        dsH = dsH.isel(time=slice(-16*8-1, None))  # only last 16 days for short case
        dsH["time"] = dsH["time"] - dsH["time"].values[0]  # reset time to start at 0
        
    ensure_dir(output_folder+"/momentum_terms_H/")
    dsH[["circulation", "RVF", "BS", "SS"]].to_netcdf(
        output_folder+f"/momentum_terms_H/momentum_terms_H_{case}.nc"
    )
    
    
    #####################################################
    ############### CARTESIAN DIAGNOSES #################
    #####################################################
    
    dsY = interp_ds(ds, params, ["u", "v", "zetav","forcing_x", "detadx", "duvhdy"])
    dsY = dsY.isel(
        time=slice(
            analysis_region[case]["focus_time_start"],
            analysis_region[case]["focus_time_end"],
        ),
        yC=slice(
            analysis_region[case]["focus_j_start"],
            analysis_region[case]["focus_j_end"],
        ),
    )
    
    if case == "short":
        dsY = dsY.isel(time=slice(-16*8, None))  # only last 16 days for short case   
    
    dsY["time"] = dsY["time"] - dsY["time"].values[0]  # reset time to start at 0

    # calculate circulation
    dsY["circulation"] = dsY.u.mean("xC")

    # calculate momentum terms
    H0 = dsY.bath.mean("xC")
    h = H0 - dsY.bath 

    dsY["BS"] = -dsY.circulation*params["R"]                       # Bottom stress
    dsY["TFS"] = (-params["gravitational_acceleration"]*dsY.detadx*dsY.bath).mean("xC")  # Topographic form stress
    dsY["MFC"] = (-dsY.duvhdy).mean("xC")                          # Momentum flux convergence
    dsY["SS"] = (dsY.forcing_x).mean("xC")                         # Surface stress
    dsY["QGPVF"] = (dsY.zetav).mean("xC")*H0 + (dsY.v*h).mean("xC")*params["f"]          # Quasi-geostrophic potential vorticity flux
    
    
    # save relevant momentum terms
    ensure_dir(output_folder+"/momentum_terms_y/")
    dsY[["circulation", "BS","TFS", "MFC", "SS", "QGPVF"]].to_netcdf(
        output_folder+f"/momentum_terms_y/momentum_terms_y_{case}.nc"
    )   


Loading configuration from ../configs/baseline_forcing/long.json
Loading configuration from ../configs/baseline_forcing/short.json


## Contrasting cases without corrugations

Next we analyze a set of contrasting simulations with a *smooth* slope (no corrugations).  
These cases are used as a baseline to isolate the role of corrugation-driven flow–topography interactions in the focus cases.

We consider four simulations:
- `long_nobumps` and `short_nobumps`: long-/short-period forcing with *along-slope* wind stress only (same periods as the focus cases),
- `long_nobumps_crosswind` and `short_nobumps_crosswind`: the same, but with an additional cross-slope wind-stress component.

All four cases are analyzed in **Cartesian (constant-$y$) coordinates**, since the smooth slope does not require depth-following interpolation for our diagnostics. For each case we:
1. interpolate the raw model output to the analysis grid (`dsY`),
2. restrict the dataset to the same focus time window and cross-slope region as used previously (based on `long`/`short`),
3. compute along-slope circulation and the associated momentum terms (surface stress, bottom stress, momentum flux convergence, and topographic form stress),
4. save both:
   - **time-mean cross-slope structure** (terms averaged over time, stored as `*y`), and
   - **time series of domain-mean terms** (terms averaged over `y`, stored as `*t`),

to compact NetCDF files for plotting and comparison.


In [4]:
for case in ["long_nobumps", "short_nobumps", "long_nobumps_crosswind", "short_nobumps_crosswind"]:
    # load data
    params = load_config(f"../configs/baseline_forcing/{case}.json")
    ds = read_raw_output(params)
    


    dsY = interp_ds(ds, params, ["u", "v", "zetav","forcing_x", "detadx", "duvhdy"])
    dsY = dsY.isel(
        time=slice(
            analysis_region[case.split("_")[0]]["focus_time_start"],
            analysis_region[case.split("_")[0]]["focus_time_end"],
        ),
        yC=slice(
            analysis_region[case.split("_")[0]]["focus_j_start"],
            analysis_region[case.split("_")[0]]["focus_j_end"],
        ),
    )
    
    if case.split("_")[0] == "short":
        dsY = dsY.isel(time=slice(-16*8, None))  # only last 16 days for short case   
    
    dsY["time"] = dsY["time"] - dsY["time"].values[0]  # reset time to start at 0



    dsY["circulation"] = dsY.u.mean("xC")
    dsY["TFS"] = (-params["gravitational_acceleration"]*dsY.detadx*dsY.bath).mean("xC")
    dsY["MFC"] = (-dsY.duvhdy).mean("xC")
    dsY["SS"] = (dsY.forcing_x).mean("xC")
    dsY["BS"] = -dsY.circulation*params["R"]
    
    # time-mean terms
    dsY["TFSy"] = dsY["TFS"].mean("time")
    dsY["MFCy"] = dsY["MFC"].mean("time")
    dsY["SSy"] = dsY["SS"].mean("time")
    dsY["BSy"] = dsY["BS"].mean("time")
    
    # y meaned terms
    dsY["TFSt"] = dsY["TFS"].mean("yC")
    dsY["MFCt"] = dsY["MFC"].mean("yC")
    dsY["SSt"] = dsY["SS"].mean("yC")
    dsY["BSt"] = dsY["BS"].mean("yC")
    
    # save relevant momentum terms
    ensure_dir(output_folder+"/momentum_terms_y/")
    dsY[["circulation", "TFSt", "MFCt", "SSt", "BSt","TFSy", "MFCy", "SSy", "BSy"]].to_netcdf(
        output_folder+f"/momentum_terms_y/momentum_terms_y_{case}.nc"
    )   

Loading configuration from ../configs/baseline_forcing/long_nobumps.json
Loading configuration from ../configs/baseline_forcing/short_nobumps.json
Loading configuration from ../configs/baseline_forcing/long_nobumps_crosswind.json
Loading configuration from ../configs/baseline_forcing/short_nobumps_crosswind.json


## Wave-based diagnostics

In this section, we compute a set of diagnostics used to compare the simulated circulation response to characteristic properties of topographic waves. 

The calculations below focus on three related quantities:
1. characteristic prograde and retrograde flow speeds inferred from the simulations,
2. the cross-slope structure of the dominant wave-like response, and
3. the dependence of maximum flow speed on forcing strength across multiple realizations.



### Arrest speed

We first estimate characteristic prograde and retrograde flow speeds directly from the simulated circulation along depth-following contours. These speeds are used as empirical measures of the maximum along-slope flow attained in each direction, and are interpreted as proxies for wave arrest speeds.

For a given depth contour (or range of contours), we extract the circulation time series and identify:
- the maximum prograde circulation attained over the analysis period, and
- the maximum retrograde circulation (in magnitude).

When `spread=True`, we additionally estimate the range of retrograde speeds across neighboring contours, providing a measure of spatial variability in the arrest behavior.

These diagnostics are computed for simulations with different corrugation wavelengths, enabling direct comparison between arrest speeds and topographic length scales.


In [5]:
def estimate_prograde_retrograde_speed(dsH,selection=slice(40,50), spread=True):
    slope = dsH.sel(j=selection).circulation
    #slope = dsH.sel(j=slice(40,50)).circulation    
    
    if spread:
        prograde = slope.max("time").mean("j").item()
        retrograde = np.abs(slope.min("time").mean("j").item())
        
        retrograde_max = np.abs(slope.min("time").min("j").item())
        retrograde_min = np.abs(slope.min("time").max("j").item())
    
    
        return retrograde, prograde, retrograde_min, retrograde_max

    else:
        retrograde = np.abs(np.min(slope))
        prograde = np.max(slope)
        
        return retrograde, prograde

In [6]:

params_22km = load_config("../configs/varying_bathymetry/half.json")
params_45km = load_config("../configs/baseline_forcing/long.json")
params_90km = load_config("../configs/varying_bathymetry/double.json")

arrest_speed_results = {}

for params, wavelength in zip([params_22km, params_45km, params_90km], [22.5, 45, 90]):
    ds = read_raw_output(params)
    dsH = prepare_dsH(ds, params, H_targets)
    
    # diagnose circulation
    dsH["circulation"] = mean_onH(dsH, variable="ui")
    
    retrograde, prograde, retrograde_min, retrograde_max = estimate_prograde_retrograde_speed(dsH)
    
    arrest_speed_results[wavelength] = {
        "retrograde": retrograde,
        "prograde": prograde,
        "retrograde_min": retrograde_min,
        "retrograde_max": retrograde_max,
    }
    
ensure_dir(output_folder+"/wave_comparison/")
json.dump(arrest_speed_results, open(output_folder+"/wave_comparison/arrest_speeds.json", "w"))

Loading configuration from ../configs/varying_bathymetry/half.json
Loading configuration from ../configs/baseline_forcing/long.json
Loading configuration from ../configs/varying_bathymetry/double.json


### Mode structure

To examine the spatial structure of the wave-like response, we analyze the cross-slope structure of sea-surface height anomalies during the long-period forcing case.

The total height field is decomposed into a mean and an anomaly, and the anomaly is separated into two time intervals corresponding to opposite phases of the forcing. By combining these two phases, we construct a composite field to isolate the wave mode, suppressing the strong topogrpahic influence.

In [7]:
params = load_config("../configs/baseline_forcing/long.json")
ds = read_raw_output(params)
ds = ds.isel(
    time=slice(
        analysis_region["long"]["focus_time_start"],
        analysis_region["long"]["focus_time_end"],
    ),
)
ds["time"] = ds["time"] - ds["time"].values[0]  # reset time to start at 0

eta = ds["h"] - ds["bath"]
etanod = eta - eta.mean(dim="xC")

etanod_neg = etanod.isel(time=slice(0,64*8))
etanod_pos = etanod.isel(time=slice(64*8,None))
etanod_pos["time"] = etanod_pos["time"] - etanod_pos["time"].isel(time=0)

mode = etanod_neg + etanod_pos 

mode.to_netcdf(output_folder+"/wave_comparison/mode_structure.nc")

Loading configuration from ../configs/baseline_forcing/long.json


## 

### Maximum speed as a function of forcing strength

Finally, we examine how the maximum prograde and retrograde circulation scales with forcing amplitude. For a set of simulations spanning a range of wind-stress strengths, we extract the maximum circulation attained at selected depth contours.

For each forcing period (short and long), multiple realizations are analyzed. The results are assembled into a tidy dataset containing:
- forcing strength,
- maximum prograde circulation, and
- maximum retrograde circulation,

as functions of depth and forcing amplitude.

These diagnostics are used to assess whether circulation increases linearly with forcing, or whether nonlinear saturation occurs.


In [8]:
depths_to_use = [H_targets[40], H_targets[45], H_targets[50]]
periods = ["short", "long"]
n_runs = 8 

# --- Pre-allocate tidy dataset ---
ds_out = xr.Dataset(
    coords=dict(
        period=("period", periods),
        run=("run", np.arange(1, n_runs + 1)),
        depth=("depth", depths_to_use),
    ),
    data_vars=dict(
        forcing_strength=(("period", "run"), np.full((len(periods), n_runs), np.nan)),
        prograde_max=(("period", "depth", "run"), np.full((len(periods), len(depths_to_use), n_runs), np.nan)),
        retrograde_max=(("period", "depth", "run"), np.full((len(periods), len(depths_to_use), n_runs), np.nan)),
    ),
)

for p_idx, p in enumerate(periods):
    for r in range(1, n_runs + 1):
        params_i = load_config(f"../configs/varying_forcing/{p}_{r:03d}.json")
        forcing = params_i["tau0"]
        ds_i = read_raw_output(params_i).isel(time=slice(-128*8, None))

        dsH_i = prepare_dsH(ds_i, params_i, depths_to_use)
        circ = mean_onH(dsH_i, variable="ui")
        dsH_i["circulation"] = circ#hanning_filter(circ, window_length=2*8)

        ds_out["forcing_strength"][p_idx, r - 1] = forcing

        for k, H in enumerate(dsH_i["depth"].values):
            retro, pro = estimate_prograde_retrograde_speed(dsH_i, selection=k, spread=False)
            ds_out["prograde_max"][p_idx, k, r - 1] = pro
            ds_out["retrograde_max"][p_idx, k, r - 1] = retro
            
ds_sorted = ds_out.copy() 
for p in periods: 
    F = ds_out["forcing_strength"].sel(period=p) 
    order = np.argsort(F.values) 
    ds_sorted["forcing_strength"].loc[dict(period=p)] = F.values[order] 
    for var in ["prograde_max", "retrograde_max"]: 
        ds_sorted[var].loc[dict(period=p)] = ds_out[var].sel(period=p).values[..., order]
        
vars_to_save = ["forcing_strength", "prograde_max", "retrograde_max"]
ds_to_write = ds_sorted[vars_to_save]

ds_to_write.to_netcdf(output_folder+"/wave_comparison/forcing_vs_arrest_speeds.nc")


Loading configuration from ../configs/varying_forcing/short_001.json
Loading configuration from ../configs/varying_forcing/short_002.json
Loading configuration from ../configs/varying_forcing/short_003.json
Loading configuration from ../configs/varying_forcing/short_004.json
Loading configuration from ../configs/varying_forcing/short_005.json
Loading configuration from ../configs/varying_forcing/short_006.json
Loading configuration from ../configs/varying_forcing/short_007.json
Loading configuration from ../configs/varying_forcing/short_008.json
Loading configuration from ../configs/varying_forcing/long_001.json
Loading configuration from ../configs/varying_forcing/long_002.json
Loading configuration from ../configs/varying_forcing/long_003.json
Loading configuration from ../configs/varying_forcing/long_004.json
Loading configuration from ../configs/varying_forcing/long_005.json
Loading configuration from ../configs/varying_forcing/long_006.json
Loading configuration from ../configs/va